In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import glob
import pickle as pkl

import pandas as pd

In [3]:
results = []
for data_file in glob.glob('../models/*.pkl'):
    with open(data_file, 'rb') as f:
        data = pkl.load(f)
        results.append(pd.DataFrame({
            'CLOSE_TIME': data['TT_id'][:,1],
            'PREDICTIONS': data['TT_y_pred']
        }))
        
        print(data['TT_y_pred'].sum())
        print(len(data['TT_y_pred']))

1066.0
100000
2960.0
99881
2930.0
99971
2980.0
99986
2700.0
99821
2559.0
99761
2504.0
99701
3020.0
99996
2932.0
99941


In [4]:
def stack_ensembles_classif(results, method='any'):
    merged_results = reduce(lambda x, y: pd.merge(x, y, on = 'CLOSE_TIME'), results)
    
    if method=='any':
        final_pred = ((merged_results.iloc[:, 1:] > 0.5).sum(axis=1) > 0).astype(float)
    if method=='voting':
        final_pred = merged_results.iloc[:, 1:].sum(axis=1)/float(merged_results.shape[1]-1)
    
    return pd.DataFrame({'CLOSE_TIME': merged_results.CLOSE_TIME, 'PREDICTIONS': final_pred})

In [9]:
stack_pred = stack_ensembles_classif(results, method='voting')
stack_pred['PREDICTIONS'].sum()

2624.2222222222226

In [10]:
# Get Target
%run targets.ipynb
file = 'ETHBTC_1m_1519496760000_1549568220000.csv'
target = get_target_1('../data/'+file)
target.shape
target.head()

(499999, 2)

,CLOSE_TIME,TARGET
0,1519496819999,1.0
1,1519496879999,0.0
2,1519496939999,0.0
3,1519496999999,1.0
4,1519497059999,0.0


In [11]:
df = pd.merge(stack_pred, target, on = ['CLOSE_TIME'])
df.head()

,CLOSE_TIME,PREDICTIONS,TARGET
0,1543586219999,0.0,0.0
1,1543586279999,0.0,1.0
2,1543586339999,0.0,0.0
3,1543586399999,0.0,0.0
4,1543586459999,0.0,1.0


In [8]:
%run utils.ipynb
eval_class_metrics(df.TARGET, df.PREDICTIONS, metrics=['auc', 'f1', 'acc', 'log_loss'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


{'acc': 0.5160429684757425,
 'auc': 0.5019486478778278,
 'f1': 0.11053145796080892,
 'log_loss': 16.715307724343152}

In [12]:
%run utils.ipynb
eval_class_metrics(df.TARGET, df.PREDICTIONS, metrics=['auc', 'f1', 'acc', 'log_loss'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


{'acc': 0.515822308703022,
 'auc': 0.5005237741568332,
 'f1': 0.044193644193644195,
 'log_loss': 16.722914125778118}